In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

### Definindo Paramêtros

In [8]:
data = 'train-mod.csv'
kval = 10
itertot = 40
sigma = 1.2
itergd = 300

### Calcula valor através da função escolhida

In [9]:
def transforminput(param, center):
    
    def rbfGaussiana(i, j, sigma):
        return np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
  
    def rbfMultiquadratica(i, j):
        return np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))/np.sum(center[j], axis=0)

    def rbfMultiquadraticaInversa(i, j):
        return np.sum(center[j], axis=0)/np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))
    
    newinput = np.zeros((len(param), len(center))).astype('float32')
    for i in range(len(param)):
        for j in range(len(center)):
            newinput[i,j] = rbfGaussiana(i, j, sigma)
    return newinput

In [10]:
def generatemodel(numparam):
    model = Sequential()
    model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]

### Tratando entrada

In [12]:
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])

alldata = np.asarray(alldata)

#dividing data

trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

###############
#normalization#
###############

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

### KMeans

In [13]:
kmean = np.zeros((kval, len(trainparamnorm[0])))
for i in range(kval):
    for j in range(len(kmean[0])):
        kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))
#looping of real algorithm
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
    print ('iteração para: ', i)
    for j in range(len(distmin)):
        #determine euclid distance
        distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
        distmin[j] = np.argmin(distall)
#search new k mean
for j in range(kval):
    clust = []
    for k in range(len(distmin)):
        if distmin[k] == j:
            clust.append(trainparamnorm[k])
    if len(clust) > 0:
        kmean[j] = np.mean(np.asarray(clust), axis=0)

iteração para:  0
iteração para:  1
iteração para:  2
iteração para:  3
iteração para:  4
iteração para:  5
iteração para:  6
iteração para:  7
iteração para:  8
iteração para:  9
iteração para:  10
iteração para:  11
iteração para:  12
iteração para:  13
iteração para:  14
iteração para:  15
iteração para:  16
iteração para:  17
iteração para:  18
iteração para:  19
iteração para:  20
iteração para:  21
iteração para:  22
iteração para:  23
iteração para:  24
iteração para:  25
iteração para:  26
iteração para:  27
iteração para:  28
iteração para:  29
iteração para:  30
iteração para:  31
iteração para:  32
iteração para:  33
iteração para:  34
iteração para:  35
iteração para:  36
iteração para:  37
iteração para:  38
iteração para:  39


In [14]:
#tranform our input
newinput = transforminput(trainparamnorm, kmean)

print(trainlabel)

[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.

In [15]:
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
600/600 [==============================] - 0s 251us/step - loss: 0.2672 - acc: 0.4017
Epoch 2/300
600/600 [==============================] - 0s 41us/step - loss: 0.2626 - acc: 0.4017
Epoch 3/300
600/600 [==============================] - 0s 49us/step - loss: 0.2584 - acc: 0.4033
Epoch 4/300
600/600 [==============================] - 0s 41us/step - loss: 0.2546 - acc: 0.4250
Epoch 5/300
600/600 [==============================] - 0s 42us/step - loss: 0.2514 - acc: 0.4950
Epoch 6/300
600/600 [==============================] - 0s 50us/step - loss: 0.2487 - acc: 0.5333
Epoch 7/300
600/600 [==============================] - 0s 40us/step - loss: 0.2462 - acc: 0.5917
Epoch 8/300
600/600 [==============================] - 0s 40us/step - loss: 0.2440 - acc: 0.6317
Epoch 9/300
600/600 [==============================] - 0s 42us/step - loss: 0.2422 - acc: 0.6183
Epoch 

600/600 [==============================] - 0s 44us/step - loss: 0.2078 - acc: 0.6567
Epoch 79/300
600/600 [==============================] - 0s 38us/step - loss: 0.2075 - acc: 0.6583
Epoch 80/300
600/600 [==============================] - 0s 38us/step - loss: 0.2073 - acc: 0.6583
Epoch 81/300
600/600 [==============================] - 0s 35us/step - loss: 0.2070 - acc: 0.6617
Epoch 82/300
600/600 [==============================] - 0s 49us/step - loss: 0.2067 - acc: 0.6633
Epoch 83/300
600/600 [==============================] - 0s 37us/step - loss: 0.2064 - acc: 0.6650
Epoch 84/300
600/600 [==============================] - 0s 57us/step - loss: 0.2062 - acc: 0.6633
Epoch 85/300
600/600 [==============================] - 0s 43us/step - loss: 0.2059 - acc: 0.6717
Epoch 86/300
600/600 [==============================] - 0s 36us/step - loss: 0.2056 - acc: 0.6750
Epoch 87/300
600/600 [==============================] - 0s 40us/step - loss: 0.2054 - acc: 0.6817
Epoch 88/300
600/600 [===========

600/600 [==============================] - 0s 40us/step - loss: 0.1917 - acc: 0.7417
Epoch 162/300
600/600 [==============================] - 0s 38us/step - loss: 0.1916 - acc: 0.7383
Epoch 163/300
600/600 [==============================] - 0s 41us/step - loss: 0.1915 - acc: 0.7383
Epoch 164/300
600/600 [==============================] - 0s 35us/step - loss: 0.1914 - acc: 0.7400
Epoch 165/300
600/600 [==============================] - 0s 40us/step - loss: 0.1913 - acc: 0.7383
Epoch 166/300
600/600 [==============================] - 0s 40us/step - loss: 0.1911 - acc: 0.7400
Epoch 167/300
600/600 [==============================] - 0s 39us/step - loss: 0.1910 - acc: 0.7383
Epoch 168/300
600/600 [==============================] - 0s 44us/step - loss: 0.1909 - acc: 0.7400
Epoch 169/300
600/600 [==============================] - 0s 39us/step - loss: 0.1908 - acc: 0.7383
Epoch 170/300
600/600 [==============================] - 0s 44us/step - loss: 0.1907 - acc: 0.7367
Epoch 171/300
600/600 [=

600/600 [==============================] - 0s 37us/step - loss: 0.1844 - acc: 0.7450
Epoch 244/300
600/600 [==============================] - 0s 39us/step - loss: 0.1844 - acc: 0.7467
Epoch 245/300
600/600 [==============================] - 0s 33us/step - loss: 0.1843 - acc: 0.7450
Epoch 246/300
600/600 [==============================] - 0s 34us/step - loss: 0.1843 - acc: 0.7467
Epoch 247/300
600/600 [==============================] - 0s 35us/step - loss: 0.1842 - acc: 0.7433
Epoch 248/300
600/600 [==============================] - 0s 46us/step - loss: 0.1841 - acc: 0.7433
Epoch 249/300
600/600 [==============================] - 0s 42us/step - loss: 0.1841 - acc: 0.7433
Epoch 250/300
600/600 [==============================] - 0s 37us/step - loss: 0.1840 - acc: 0.7433
Epoch 251/300
600/600 [==============================] - 0s 52us/step - loss: 0.1840 - acc: 0.7450
Epoch 252/300
600/600 [==============================] - 0s 36us/step - loss: 0.1839 - acc: 0.7450
Epoch 253/300
600/600 [=

In [17]:
newinputtest = transforminput(testparamnorm, kmean)

In [18]:
lifeprob = mod.predict(newinputtest)

### Armazenando os valores previstos

In [19]:
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.

### Calculando a acurácia

In [20]:
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))

In [21]:
errorabs = abs(score-len(lifeprob))
error = np.sum(errorabs/len(testlabel), axis=0)

In [22]:
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

In [23]:
for i in range(len(testlabel)):
    print (lifeprob[i], "-", testlabel[i])

[0.22433826] - 0.0
[0.64833766] - 1.0
[0.3051982] - 1.0
[0.2452465] - 0.0
[0.21083316] - 0.0
[0.2727098] - 0.0
[0.687762] - 1.0
[0.30880332] - 0.0
[0.21557328] - 1.0
[0.79820645] - 1.0
[0.21027026] - 0.0
[0.6854709] - 1.0
[0.66025925] - 0.0
[0.2631445] - 0.0
[0.22692898] - 0.0
[0.34931123] - 0.0
[0.55838263] - 0.0
[0.57859516] - 1.0
[0.21129376] - 0.0
[0.44424275] - 1.0
[0.9692399] - 1.0
[0.47998115] - 1.0
[0.56689066] - 1.0
[0.3219973] - 0.0
[0.23287347] - 0.0
[0.23271391] - 0.0
[0.5245637] - 1.0
[0.2319493] - 0.0
[0.283853] - 1.0
[0.49695235] - 0.0
[0.21634823] - 0.0
[0.2322109] - 0.0
[0.17135587] - 0.0
[0.5675175] - 1.0
[0.6640894] - 1.0
[0.2545994] - 0.0
[0.6648316] - 0.0
[0.16376525] - 0.0
[0.6439295] - 1.0
[0.5644032] - 1.0
[0.29038763] - 1.0
[0.2414096] - 1.0
[0.2588405] - 0.0
[0.31576672] - 0.0
[0.52468807] - 0.0
[0.17135587] - 0.0
[0.6024847] - 1.0
[0.2364308] - 0.0
[0.2968725] - 0.0
[0.1667192] - 0.0
[0.45350692] - 0.0
[0.25639182] - 0.0
[0.5834761] - 0.0
[0.17189875] - 0.0
[

In [25]:
print ("Accuracy: ", accbin, "Brier score:", brierscore[0])
print ("Error: ", error)

Accuracy:  0.7894736842105263 Brier score: 0.16372675
Error:  0.21052631578947367
